# A\* Global Path Planning Algorithm
A\*算法是一种全局路径规划算法，被广泛应用与多种场合，包括车辆行驶路径寻找、游戏中的寻路等. 该算法是启发式搜索算法，寻找一条从起点到终点的路径. 

从本质上来说，A\*算法是Dijkstra算法与广度优先搜索算法(BFS algorithm)的推广. 这三种算法都是将地图抽象成一张图(Graph)，利用图路径搜索算法来得到找到路径.

## 1 - A\*算法简介
### 1.1 - 代价函数
A\*算法类似于Dijkstra算法，能够寻找出一条从起点至终点，具有"最小代价"的路径. A\*算法与其他路径搜索算法不同的一点在于：对每个结点(node), A\*算法使用一个代价函数$f(n)$来表示路径的总代价. 这个函数定义为：
$$f(n) = g(n) + h(n)$$
其中：
1. $f(n)$ = 从起始节点, 通过结点n, 到达终点的路径的估计代价
2. $g(n)$ = 从起点到达结点n的代价
3. $h(h)$ = 结点n到终点的代价估计

三者中，$h(n)$作为代价方程中的"启发"部分(Heuristic part of cost function), 通常是无法得到准确值的. 因此我们需要对其进行定义. 常见的$h(n)$有两种:
* $h(n)$ = 从结点n到终点的**曼哈顿距离(Manhattan Distance)**, 这种定义方式使用地最普遍;
* $h(n)$ = 0, 此时，A\*算法就退化成了Dijkstra算法. Dijkstra算法能够保证找到最短路径.

$h(n)$也可以采用别的函数，但是需要注意的是，$h(n)$的定义必须是admissible, 不能高估结点n到终点的代价.

我们采用曼哈顿距离来定义$h(n)$, 曼哈顿距离的数学公式为:
$$h_{manhattan} = |x_{start} - x_{end}| + |y_{start} - y_{end}|$$

### 1.2 - 寻路算法
我们使用一个例子来对A\*算法进行说明.

在开始之前需要先导入必要的包:

In [100]:
import numpy as np
import matplotlib.pyplot as plt

我们首先构建一个简单的地图：

In [101]:
tm = np.array(['############################################################',
'#..........................................................#',
'#.............................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#.......S.....................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#.............................#............................#',
'#######.#######################################............#',
'#....#........#............................................#',
'#....#........#............................................#',
'#....##########............................................#',
'#..........................................................#',
'#..........................................................#',
'#..........................................................#',
'#..........................................................#',
'#..........................................................#',
'#...............................##############.............#',
'#...............................#........E...#.............#',
'#...............................#............#.............#',
'#...............................#............#.............#',
'#...............................#............#.............#',
'#...............................###########..#.............#',
'#..........................................................#',
'#..........................................................#',
'############################################################'])

地图中，'.'号表示的是可以前进的位置，'#'为不可前进的位置，即障碍物, 'S'为起点，'E'为终点. 由于python中无法改变string的某个元素值，因此我们还需要单独定义一个test_map来存储搜索时的地图.

In [102]:
test_map = []

我们要从S点移动到E点，在这个过程中，每走一步(即移动一个方格)，准备进行下一步时，都会产生一个状态: 可走 or 不可走. 我们需要构建两个集合closed与open，前者存放已经被估算的节点，后者存放将要被估算的节点. 另外还需要一个列表来记录每个节点的$f(n)$值. 

整个寻路过程可以被分为以下几个步骤：
1. 将起点S放入open set中;
2. 查看与起点S相邻的节点，把其中可走的节点加入到open set中，并把起点A设置为这些节点的父节点;
3. 从open set中移除S，并将S加入closed set;
4. 在open set中选取f(n)值最小的节点BST，放入closed set中;
5. 检查BST周围所有可走的点，把可走的点放入open set中;
6. 如果可走的点已经在open set中，则检查BST作为父节点的g(n)是否比之前在open set时小，如果小，则替换成新的父节点;
7. 不断重复上边的过程，直到目标节点在closed set中，或者open set为空;
8. 目标节点沿着父节点递归，直到找到S，至此找到完整的路径.

为了使用方便，我们首先定义每个节点的数据类型:

In [103]:
class Node_Elem:
    """
    open与closed列表中存储的元素类型
    """
    def __init__(self, parent, x, y, dist):
        '''
        一个节点中有四个元素，分别为"父节点"、"x坐标"、"y坐标"、"从起点到当前点的距离"
        '''
        self.parent = parent
        self.x = x
        self.y = y
        self.dist = dist

另外，我们还需要对算法实现过程中的一些数据进行初始化:

In [104]:
## 初始化open set与closed set
openSet = []
closedSet = []
path = []

## 通过函数获取起点、终点和地图的相关信息
def before_searching(s_x, s_y, e_x, e_y, w = 60, h = 30):
    return s_x, s_y, e_x, e_y, w, h

## 将tm地图导入test_mao
def tm_to_test_map():
    for line in tm:
        test_map.append(list(line))
        
## 得到起点与终点
def get_start_XY():
    return get_symbol_XY('S')

def get_end_XY():
    return get_symbol_XY('E')

def get_symbol_XY(str):
    for y, line in enumerate(test_map):
        try:
            x = line.index(str)
        except:
            continue
        else:
            break
    return x, y

## 得到地图的长宽
def get_map_wh():
    w = len(test_map[0])
    h = len(test_map)
    return w, h

将地图转存到test_map中，并获取必要的参数:

In [105]:
test_map = []
## 转存
tm_to_test_map()
print(len(test_map))
## 得到起点与终点
s_x, s_y = get_start_XY()
e_x, e_y = get_end_XY()

print('起点：')
print(s_x, s_y)

print('终点：')
print(e_x, e_y)

## 得到地图的长宽
w, h = get_map_wh()
print('地图长宽：')
print(w, h)

31
起点：
8 5
终点：
41 23
地图长宽：
60 31


接下来就是根据算法计算路径了:

In [107]:
## 为了方便，我们将A*算法实现为一个类
class A_Star:
    def __init__(self, s_x, s_y, e_x, e_y, test_map, w=60, h=30):
        self.s_x = s_x
        self.s_y = s_y
        self.e_x = e_x
        self.e_y = e_y
        
        self.width = w
        self.height = h
        
        self.open = []
        self.close = []
        self.path = []
        
        self.test_map = test_map
        
    ## 路径查找
    def find_path(self):
        # 构建起始节点
        p = Node_Elem(None, self.s_x, self.s_y, 0.0)
    
        # 循环体
        while True:
            # 扩展当前节点
            self.extend_round(p)
        
            # 如果open set为空，则不存在路径，输出该信息
            if not self.open:
                print('没有路径存在!请检查地图是否正确\n')
                return -1
        
            # 获取open set中f(n)最小的点
            idx, p = self.get_best()
            
            # 如果p就是目标节点，则找到路径并生成路径
            if self.is_target(p):
                self.make_path(p)
                print('已经找到路径!')
                return 1
            
            # 如果p不是目标节点，则将该节点压如close set中，并从open set中删除该列表
            self.close.append(p)
            del self.open[idx]
        
    ## 生成路径
    def make_path(self, p):
        while p:
            self.path.append((p.x, p.y))
            p = p.parent
        
    ## 判断是否是目标点
    def is_target(self, pt):
        return pt.x == self.e_x and pt.y == self.e_y

    ## 获取open set中f(n)最小的点
    def get_best(self):
        best = None
        bv = 1000000000    # 该值随着地图的扩张而增大
        bi = -1         # 记录点在open set中的位置
    
        for idx, i in enumerate(self.open):
            value = self.get_dist(i)   # 获取f(n)值
            if value < bv:
                best = i
                bv = value
                bi = idx
            
        return bi, best

    ## 获取f(n)值
    def get_dist(self, pt):
        '''
        这一部分是A\*算法的精华所在，即f(n) = g(n) + h(n)
        这里采用Manhattan距离作为h(n)
        '''
        g = pt.dist
        h = abs(pt.x - self.e_x) + abs(pt.y - self.e_y)
        
        return g + h

    ## 扩展节点
    def extend_round(self, pt):
        # 八方向扩展
        xs = (-1, 0, 1, -1, 1, -1, 0, 1)
        ys = (-1,-1,-1,  0, 0,  1, 1, 1)
        
        for x, y in zip(xs, ys):
            new_x, new_y = x + pt.x, y + pt.y
            # 若是无效或者不可行走的区域，则忽略
            if not self.is_valid_coord(new_x, new_y):
                continue
                
            # 构造新的节点
            node = Node_Elem(pt, new_x, new_y, pt.dist + self.get_cost(pt.x, pt.y, new_x, new_y))
        
            # 若新节点在closed set，则忽略
            if self.node_in_close(node):
                continue
            # 若新节点不在closed set
            i = self.node_in_open(node)
            if i != -1:
                # 新节点在open set
                if self.open[i].dist > node.dist:
                    # 现在的路径到open[i]的距离小于原先的距离
                    self.open[i].parent = pt
                    self.open[i].dist = node.dist
                continue
        
            # 加入open set中
            self.open.append(node)
        
    ## 获取前进的代价
    def get_cost(self, x1, x2, y1, y2):
        # 上下直走代价为1, 斜走代价为1.4
        if x1 == x2 or y1 == y2:
            return 1.0
        return 1.4

    ## 判断是否在closed set中
    def node_in_close(self, node):
        for i in self.close:
            if node.x == i.x and node.y == i.y:
                return True
        return False

    ## 判断是否在open set中
    def node_in_open(self, node):
        for i, n in enumerate(self.open):
            if node.x == n.x and node.y == n.y:
                return i
        return -1

    ## 是否是障碍物
    def is_valid_coord(self, x, y):
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return False
        return self.test_map[y][x] != '#'
    
    ## 得到搜索路径
    def get_searched(self):
        l = []
        for i in self.open:
            l.append((i.x, i.y))
        for i in self.close:
            l.append((i.x, i.y))
        return l

检验是否有bug

In [109]:
a_star = A_Star(s_x, s_y, e_x, e_y, test_map, w, h)
a_star.find_path()
searched = a_star.get_searched()
path = a_star.path
print(path)

已经找到路径!
[(41, 23), (41, 24), (42, 25), (43, 26), (44, 27), (45, 28), (46, 27), (46, 26), (46, 25), (46, 24), (46, 23), (46, 22), (45, 21), (44, 20), (43, 19), (42, 18), (43, 17), (44, 16), (45, 15), (46, 14), (47, 13), (46, 12), (45, 12), (44, 12), (43, 12), (42, 12), (41, 12), (40, 11), (39, 10), (38, 9), (37, 8), (36, 7), (35, 6), (34, 5), (33, 4), (32, 3), (31, 2), (30, 1), (29, 2), (28, 3), (27, 4), (26, 5), (25, 6), (24, 7), (23, 8), (22, 9), (21, 10), (20, 11), (19, 12), (18, 12), (17, 12), (16, 12), (15, 12), (14, 11), (13, 10), (12, 9), (11, 8), (10, 7), (9, 6), (8, 5)]


定义一个函数，将找到的路径打印出来

In [110]:
## 打印路径
def print_road(map, path):
    for pt in path:
        if map[pt[1]][pt[0]] != 'S' or map[pt[1]][pt[0]] != 'E':
            map[pt[1]][pt[0]] = 'o'
        
    for line in map:
        print(''.join(line))
    
    return 1

print_road(a_star.test_map, path)

############################################################
#.............................o............................#
#............................o#o...........................#
#...........................o.#.o..........................#
#..........................o..#..o.........................#
#.......o.................o...#...o........................#
#........o...............o....#....o.......................#
#.........o.............o.....#.....o......................#
#..........o...........o......#......o.....................#
#...........o.........o.......#.......o....................#
#............o.......o........#........o...................#
#.............o.....o.........#.........o..................#
#..............ooooo..........#..........oooooo............#
#######.#######################################o...........#
#....#........#...............................o............#
#....#........#..............................o.............#
#....##########.........

1